In [ ]:
import torch
from torch.nn.utils import prune

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from calculate_mask import calculate_mask
from inverse_hessian import inverse_hessian
from input_prehooks import put_input_hooks

In [ ]:
%%capture
import torch
from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = load_dataset('c4', 'en', streaming=True)
model_text = "facebook/opt-125m"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_text)

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained(model_text, output_attentions=True, output_hidden_states=True)

generator = pipeline('text-generation', model=model_text)

calibration_data = []
for i, data in enumerate(iter(dataset['train'])):
    if i > 128:
        break
    tokenized = tokenizer.encode(data['text'], return_tensors="pt", padding="max_length", truncation=True, max_length=768)
    calibration_data.append(tokenized)
calibration_data = torch.squeeze(torch.stack(calibration_data)).to(device=device)